In [ ]:
import numpy as np
import librosa
import matplotlib.pyplot as plt

from song_db_universal import songs

In [ ]:
SAMPLE_RATE = 22050
refs, queries = songs.get_snips_for_artist(audio_len = 1*SAMPLE_RATE, "taylorswift")
print(refs.shape, queries.shape)

In [ ]:
def preprocess(M):
    # Absolute value, log and average across time frames
    return np.average(np.log(1+1000000*np.absolute(M)),axis = 1)

In [ ]:
# Compute CQT and preprocess
refCQT = []
queryCQT = []
for ref in refs:
    refCQT.append(preprocess(librosa.cqt(np.asfarray(ref, dtype='float32'), sr = 22050, fmin=130.81, n_bins=121, bins_per_octave=24, hop_length=96)))
for query in queries: 
    queryCQT.append(preprocess(librosa.cqt(np.asfarray(query, dtype='float32'), sr = 22050, fmin=130.81, n_bins=121, bins_per_octave=24, hop_length=96)))
refCQT = np.array(refCQT)
queryCQT = np.array(queryCQT)

In [ ]:
print(refCQT.shape, queryCQT.shape)

In [ ]:
# Separate into training and testing datasets

training_p = 0.6
ref_p = int(training_p*refs.shape[0])
query_p = int(training_p*queries.shape[0])
print (ref_p, query_p)

In [ ]:
X = np.concatenate((refCQT, queryCQT), axis=0)
y = np.array([0]*refs.shape[0] + [1]*queries.shape[0])

X_train = np.concatenate((refCQT[:ref_p], queryCQT[:query_p]), axis=0)
X_test = np.concatenate((refCQT[ref_p:], queryCQT[query_p:]), axis=0)
y_train = np.array([0]*ref_p + [1]*query_p)
y_test = np.array([0]*(refs.shape[0]-ref_p) + [1]*(queries.shape[0]-query_p))
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
# SVC Model
from sklearn.svm import SVC, LinearSVC
clf = SVC()
clf.fit(X_train, y_train)

In [ ]:
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

In [ ]:
# Project features to 2-D plot
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X)

In [ ]:
# Scatter plot of all data points
plt.figure()
plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=y, zorder=10, cmap=plt.cm.Paired,
                edgecolor='k', s=20)
plt.xlabel('1st Component')
plt.ylabel('2nd Component')
plt.title('SVC, audio_len = 1s')
plt.show()